##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Maker による音声認識モデルの再トレーニング


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/speech_recognition"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/speech_recognition.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/speech_recognition.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub で表示</a>
</td>
  <td> ノートブックをダウンロード</td>
</table>

この colab ノートブックでは、[TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) を使用して、1 秒間の音声サンプルから発話された単語と短いフレーズを分類できる、音声認識モデルをトレーニングする方法について説明します。Model Maker ライブラリは転移学習を使用して、新しいデータセットで既存の TensorFlow モデルを再トレーニングします。これにより、トレーニングに必要なサンプルデータと時間が少なくなります。

既定では、このノートブックは、[音声認識データセット](https://www.tensorflow.org/datasets/catalog/speech_commands) ("up"、"down"、"left"、"right" など) を使用して、モデル ([TFJS Speech Command Recognizer](https://github.com/tensorflow/tfjs-models/tree/master/speech-commands#speech-command-recognizer) の BrowserFft) を再トレーニングします。次に、モバイルデバイスまたは組み込みシステム (Raspberry Pi など) で実行できる TFLite モデルがエクスポートされます。トレーニング済みのモデルも TensorFlow SavedModel としてエクスポートされます。

また、このノートブックは、ZIP ファイルで Colab にアップロードされる、WAV ファイルのカスタムデータセットを許可するように設計されています。各クラスのサンプルが多いほど、精度が上がります。転移学習プロセスでは、トレーニング済みのモデルから特徴埋め込みを使用するため、各クラスのサンプル数が 20 ～ 30 件しかなくても、かなり精度が高いモデルを実現できます。

**注意:** トレーニングするモデルは、1 秒間のサンプルの音声認識向けに最適化されています。異なる種類の音楽の検出のように、より一般的な音声分類を実行する場合は、[この Colab に従い、音声分類器を再トレーニング](https://colab.sandbox.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/audio_classification.ipynb)することをお勧めします。

既定の音声データセットでこのノートブックを実行する場合は、Colab ツールバーで **[Runtime] &gt; [Run all]** をクリックして、すべて実行できます。ただし、独自のデータセットを使用する場合は、 [データセットの準備](#scrollTo=cBsSAeYLkc1Z)に進み、その手順に従ってください。


### 必要なパッケージのインポート


音声の操作、再生、視覚化を行うには、TensorFlow、TFLite Model Maker、一部のモジュールが必要です。

In [ ]:
!sudo apt -y install libportaudio2
!pip install tflite-model-maker

In [ ]:
import os
import glob
import random
import shutil

import librosa
import soundfile as sf
from IPython.display import Audio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tflite_model_maker as mm
from tflite_model_maker import audio_classifier
from tflite_model_maker.config import ExportFormat

print(f"TensorFlow Version: {tf.__version__}")
print(f"Model Maker Version: {mm.__version__}")

## データセットを準備する

既定の音声データセットでトレーニングするには、次のコードすべてをそのまま実行します。

ただし、独自の音声データセットでトレーニングする場合は、次の手順に従います。

**注意:** トレーニングするモデルでは、44.1 kHz で約 1 秒間の音声が入力データとして想定されています。Model Maker によって、自動リサンプリングが実行され、データセットがトレーニングされます。このため、44.1 kHz 以外のサンプリングレートの場合、データセットをリサンプリングする必要はありません。ただし、音声サンプルが 1 秒を超える場合は、複数の 1 秒間のチャンクに分割され、最後のチャンクが 1 秒未満の場合は、その最後のチャンクが破棄されます。

1. データセットの各サンプルは **約 1 秒間の WAV ファイル形式**です。すべての WAV ファイルを含んだ ZIP ファイルを作成します。この ZIP ファイルでは、各分類が別個のサブフォルダに整理されています。たとえば、音声コマンド「yes」の各サンプルは、「yes」サブフォルダに格納されます。クラスが 1 つしかない場合でも、クラス名がディレクトリ名になっているサブディレクトリにそのサンプルを保存する必要があります。(このスクリプトでは、データセットがトレーニング/検証/テストセットに分割されていない状態で、自動的に分割が実行されることを想定しています。)
2. 左側のパネルで **[Files]** タブをクリックし、ZIP ファイルをドラッグアンドドロップしてアップロードします。
3. 次のドロップダウンオプションを使用して、**`use_custom_dataset`** を True に設定します。
4. [カスタム音声データセットの準備](#scrollTo=EobYerLQkiF1)に進み、ZIP ファイル名とデータセットディレクトリ名を指定します。


In [ ]:
use_custom_dataset = False #@param ["False", "True"] {type:"raw"}

### 背景ノイズデータセットの生成

既定の音声データセットか、カスタムデータセットのどちらを使用している場合でも、モデルで音声とその他のノイズ (無音を含む) を識別できるように、良質の背景ノイズのセットを用意する必要があります。

次の背景サンプルは、1 分間以上の WAV ファイルで提供されているため、小さい 1 秒間のサンプルに分割し、テストデータセット用に一部を予約できるようにする必要があります。いくつかの異なるサンプルソースを組み合わせ、背景ノイズと無音の包括的なセットを作成します。

In [ ]:
tf.keras.utils.get_file('speech_commands_v0.01.tar.gz',
                        'http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz',
                        cache_dir='./',
                        cache_subdir='dataset-speech',
                        extract=True)
tf.keras.utils.get_file('background_audio.zip',
                        'https://storage.googleapis.com/download.tensorflow.org/models/tflite/sound_classification/background_audio.zip',
                        cache_dir='./',
                        cache_subdir='dataset-background',
                        extract=True)


**注意:** 新しいバージョンが利用可能ですが、ダウンロードサイズが小さい v0.01 音声コマンドデータセットを使用します。v0.01 には 30 コマンドが含まれていますが、v0.02 ではさらに 5 コマンド ("backward"、"forward"、"follow"、"learn"、"visual") が追加されています。

In [ ]:
# Create a list of all the background wav files
files = glob.glob(os.path.join('./dataset-speech/_background_noise_', '*.wav'))
files = files + glob.glob(os.path.join('./dataset-background', '*.wav'))

background_dir = './background'
os.makedirs(background_dir, exist_ok=True)

# Loop through all files and split each into several one-second wav files
for file in files:
  filename = os.path.basename(os.path.normpath(file))
  print('Splitting', filename)
  name = os.path.splitext(filename)[0]
  rate = librosa.get_samplerate(file)
  length = round(librosa.get_duration(filename=file))
  for i in range(length - 1):
    start = i * rate
    stop = (i * rate) + rate
    data, _ = sf.read(file, start=start, stop=stop)
    sf.write(os.path.join(background_dir, name + str(i) + '.wav'), data, rate)

### 音声コマンドデータセットの準備

音声コマンドデータセットをダウンロードしたので、モデルのクラス数を減らす必要があります。

このデータセットには、30 以上の音声コマンド分類が含まれ、そのほとんどに 2,000 サンプル以上が含まれています。ここでは転移学習を使用しているため、それほど多くのサンプルは必要ありません。そこで、次のコードで次の処理を実行します。

- 使用する分類を指定し、それ以外を削除する。
- トレーニング用に各クラスにつき 150 サンプルだけ保持する (転移学習が少ないデータセットでも機能することを証明し、トレーニング時間を減らすため)。
- テストデータセット用に別のディレクトリを作成し、後から簡単に推論を実行できるようにする。

In [ ]:
if not use_custom_dataset:
  commands = [ "up", "down", "left", "right", "go", "stop", "on", "off", "background"]
  dataset_dir = './dataset-speech'
  test_dir = './dataset-test'

  # Move the processed background samples
  shutil.move(background_dir, os.path.join(dataset_dir, 'background'))   

  # Delete all directories that are not in our commands list
  dirs = glob.glob(os.path.join(dataset_dir, '*/'))
  for dir in dirs:
    name = os.path.basename(os.path.normpath(dir))
    if name not in commands:
      shutil.rmtree(dir)

  # Count is per class
  sample_count = 150
  test_data_ratio = 0.2
  test_count = round(sample_count * test_data_ratio)

  # Loop through child directories (each class of wav files)
  dirs = glob.glob(os.path.join(dataset_dir, '*/'))
  for dir in dirs:
    files = glob.glob(os.path.join(dir, '*.wav'))
    random.seed(42)
    random.shuffle(files)
    # Move test samples:
    for file in files[sample_count:sample_count + test_count]:
      class_dir = os.path.basename(os.path.normpath(dir))
      os.makedirs(os.path.join(test_dir, class_dir), exist_ok=True)
      os.rename(file, os.path.join(test_dir, class_dir, os.path.basename(file)))
    # Delete remaining samples
    for file in files[sample_count + test_count:]:
      os.remove(file)

### カスタムデータセットの準備

独自の音声データセットでモデルをトレーニングする場合は、ZIP の WAV ファイルとしてサンプルをアップロード ([上記を参照](#scrollTo=cBsSAeYLkc1Z)) し、次の変数を修正してデータセットを指定する必要があります。

In [ ]:
if use_custom_dataset:
  # Specify the ZIP file you uploaded:
  !unzip YOUR-FILENAME.zip
  # Specify the unzipped path to your custom dataset
  # (this path contains all the subfolders with classification names):
  dataset_dir = './YOUR-DIRNAME'

上記のファイル名とパスを変更した後、カスタムデータセットでモデルをトレーニングできます。Colab ツールバーで **[Runtime] &gt; [Run all]** を選択し、ノートブック全体を実行します。

次のコードは、新しい背景ノイズサンプルをデータセットに統合し、すべてのサンプルの部分を分離して、テストセットを作成します。

In [ ]:
def move_background_dataset(dataset_dir):
  dest_dir = os.path.join(dataset_dir, 'background')
  if os.path.exists(dest_dir):
    files = glob.glob(os.path.join(background_dir, '*.wav'))
    for file in files:
      shutil.move(file, dest_dir)
  else:
    shutil.move(background_dir, dest_dir)

In [ ]:
if use_custom_dataset:
  # Move background samples into custom dataset
  move_background_dataset(dataset_dir)

  # Now we separate some of the files that we'll use for testing:
  test_dir = './dataset-test'
  test_data_ratio = 0.2
  dirs = glob.glob(os.path.join(dataset_dir, '*/'))
  for dir in dirs:
    files = glob.glob(os.path.join(dir, '*.wav'))
    test_count = round(len(files) * test_data_ratio)
    random.seed(42)
    random.shuffle(files)
    # Move test samples:
    for file in files[:test_count]:
      class_dir = os.path.basename(os.path.normpath(dir))
      os.makedirs(os.path.join(test_dir, class_dir), exist_ok=True)
      os.rename(file, os.path.join(test_dir, class_dir, os.path.basename(file)))
    print('Moved', test_count, 'images from', class_dir)

### サンプルの再生

データセットが正しいことを確認するために、テストセットからランダムサンプルを再生します。

In [ ]:
def get_random_audio_file(samples_dir):
  files = os.path.abspath(os.path.join(samples_dir, '*/*.wav'))
  files_list = glob.glob(files)
  random_audio_path = random.choice(files_list)
  return random_audio_path

def show_sample(audio_path):
  audio_data, sample_rate = sf.read(audio_path)
  class_name = os.path.basename(os.path.dirname(audio_path))
  print(f'Class: {class_name}')
  print(f'File: {audio_path}')
  print(f'Sample rate: {sample_rate}')
  print(f'Sample length: {len(audio_data)}')

  plt.title(class_name)
  plt.plot(audio_data)
  display(Audio(audio_data, rate=sample_rate))

In [ ]:
random_audio = get_random_audio_file(test_dir)
show_sample(random_audio)

## モデルを定義する

Model Maker を使用してモデルを再トレーニングするときには、モデル仕様の定義から始める必要があります。仕様では、新しいモデルが、新しいクラスの学習を開始するために特徴埋め込みを抽出する基本モデルが定義されます。この音声認識器の仕様は、トレーニング済みの [TFJS の BrowserFft モデル](https://github.com/tensorflow/tfjs-models/tree/master/speech-commands#speech-command-recognizer)に基づいています。

モデルでは、44.1 kHz、1 秒以下の音声サンプルが入力として想定されています。正確なサンプルの長さは 44034 フレームです。

トレーニングデータセットでは、リサンプリングを実行する必要がありません。Model Maker で実行されます。ただし、後から推論を実行するときには、入力が想定された形式と一致していることを確認してください。

ここで必要な手順は、[`BrowserFftSpec`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/BrowserFftSpec) を初期化するだけです。


In [ ]:
spec = audio_classifier.BrowserFftSpec()

## データセットの読み込み 

モデル仕様に従って、データセットを読み込む必要があります。Model Maker には、[`DataLoader`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/DataLoader) API があります。これにより、フォルダからデータセットが読み込まれ、モデル仕様の想定された形式であることが保証されます。

一部のファイルはすでに個別のディレクトリに移動され、予約されています。これで、後から簡単に推論を実行できます。ここでは、各分割 (トレーニングセット、検証セット、テストセット) の `DataLoader` を作成します。

#### 音声コマンドデータセットの読み込み


In [ ]:
if not use_custom_dataset:
  train_data_ratio = 0.8
  train_data = audio_classifier.DataLoader.from_folder(
      spec, dataset_dir, cache=True)
  train_data, validation_data = train_data.split(train_data_ratio)
  test_data = audio_classifier.DataLoader.from_folder(
      spec, test_dir, cache=True)

#### カスタムデータセットの読み込み

**注意:** 特に、データセットをリサンプリングする必要があるときには、トレーニングを高速にするために、`cache=True` に設定することが重要です。ただし、データを格納するために必要な RAM の量が増えます。非常に大きいカスタムデータセットを使用する場合は、キャッシュが RAM 容量を超える可能性があります。

In [ ]:
if use_custom_dataset:
  train_data_ratio = 0.8
  train_data = audio_classifier.DataLoader.from_folder(
      spec, dataset_dir, cache=True)
  train_data, validation_data = train_data.split(train_data_ratio)
  test_data = audio_classifier.DataLoader.from_folder(
      spec, test_dir, cache=True)


## モデルのトレーニング


Model Maker [`create()`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/create) 関数を使用して、モデル仕様とトレーニングデータセットに基づいてモデルを作成し、トレーニングを開始します。

カスタムデータセットを使用している場合は、必要に応じて、トレーニングセットのサンプル数に合わせてバッチサイズを変更できます。

**注意:** 最初のエポックではキャッシュを作成する必要があるため、最初のエポックは時間がかかります。 

In [ ]:
# If your dataset has fewer than 100 samples per class,
# you might want to try a smaller batch size
batch_size = 25
epochs = 25
model = audio_classifier.create(train_data, spec, validation_data, batch_size, epochs)

## モデルパフォーマンスの確認

上記のトレーニング出力で精度/損失に問題がないように思われても、モデルがまだ確認していないテストデータを使用して、モデルを実行することが重要です。これは `evaluate()` メソッドで実行できます。

In [ ]:
model.evaluate(test_data)

### 混同行列の表示

このような分類モデルをトレーニングするときには、[混同行列](https://en.wikipedia.org/wiki/Confusion_matrix)の調査が役に立ちます。混同行列では、テストデータの各分類に対する分類器のパフォーマンスを視覚的に詳細に表示できます。

In [ ]:
def show_confusion_matrix(confusion, test_labels):
  """Compute confusion matrix and normalize."""
  confusion_normalized = confusion.astype("float") / confusion.sum(axis=1)
  sns.set(rc = {'figure.figsize':(6,6)})
  sns.heatmap(
      confusion_normalized, xticklabels=test_labels, yticklabels=test_labels,
      cmap='Blues', annot=True, fmt='.2f', square=True, cbar=False)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

confusion_matrix = model.confusion_matrix(test_data)
show_confusion_matrix(confusion_matrix.numpy(), test_data.index_to_label)

## モデルをエクスポートする

最後のステップでは、モバイル/組み込みデバイスで実行できるようにモデルを TensorFlow Lite 形式にエクスポートし、別の場所で実行できるように [SavedModel 形式](https://www.tensorflow.org/guide/saved_model)にエクスポートします。

Model Maker から `.tflite`ファイルをエクスポートすると、推論中に、後から役立つさまざまな詳細情報を説明する[モデルメタデータ](https://www.tensorflow.org/lite/inference_with_metadata/overview)が含まれます。分類ラベルファイルのコピーも含まれるため、別の `labels.txt` ファイルは必要ありません (次のセクションでは、このメタデータを使用して、推論を実行する方法を示します)。

In [ ]:
TFLITE_FILENAME = 'browserfft-speech.tflite'
SAVE_PATH = './models'

In [ ]:
print(f'Exporing the model to {SAVE_PATH}')
model.export(SAVE_PATH, tflite_filename=TFLITE_FILENAME)
model.export(SAVE_PATH, export_format=[mm.ExportFormat.SAVED_MODEL, mm.ExportFormat.LABEL])

## TF Lite モデルを使用した推論の実行

これで、TFLite モデルをデプロイし、サポートされている[推論ライブラリ](https://www.tensorflow.org/lite/guide/inference)または新しい [TFLite AudioClassifier Task API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/audio_classifier) を使用して実行できます。次のコードは、Python で `.tflite` モデルを使用して、推論を実行する方法を示します。

In [ ]:
# This library provides the TFLite metadata API
! pip install -q tflite_support

In [ ]:
from tflite_support import metadata
import json

def get_labels(model):
  """Returns a list of labels, extracted from the model metadata."""
  displayer = metadata.MetadataDisplayer.with_model_file(model)
  labels_file = displayer.get_packed_associated_file_list()[0]
  labels = displayer.get_associated_file_buffer(labels_file).decode()
  return [line for line in labels.split('\n')]

def get_input_sample_rate(model):
  """Returns the model's expected sample rate, from the model metadata."""
  displayer = metadata.MetadataDisplayer.with_model_file(model)
  metadata_json = json.loads(displayer.get_metadata_json())
  input_tensor_metadata = metadata_json['subgraph_metadata'][0][
          'input_tensor_metadata'][0]
  input_content_props = input_tensor_metadata['content']['content_properties']
  return input_content_props['sample_rate']

実際のサンプルでのモデルのパフォーマンスを観察するには、次のコードブロックを繰り返し実行します。毎回、新しいテストサンプルが取得され、そのサンプルで推論が実行されます。次の音声サンプルを聴くことができます。

In [ ]:
# Get a WAV file for inference and list of labels from the model
tflite_file = os.path.join(SAVE_PATH, TFLITE_FILENAME)
labels = get_labels(tflite_file)
random_audio = get_random_audio_file(test_dir)

# Ensure the audio sample fits the model input
interpreter = tf.lite.Interpreter(tflite_file)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_size = input_details[0]['shape'][1]
sample_rate = get_input_sample_rate(tflite_file)
audio_data, _ = librosa.load(random_audio, sr=sample_rate)
if len(audio_data) < input_size:
  audio_data.resize(input_size)
audio_data = np.expand_dims(audio_data[:input_size], axis=0)

# Run inference
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], audio_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

# Display prediction and ground truth
top_index = np.argmax(output_data[0])
label = labels[top_index]
score = output_data[0][top_index]
print('---prediction---')
print(f'Class: {label}\nScore: {score}')
print('----truth----')
show_sample(random_audio)

## TF Lite モデルのダウンロード

これで、TF Lite モデルをモバイルデバイスまたは組み込みデバイスにデプロイできます。ラベルファイルをダウンロードする必要はありません。上記の推論の例のように、ラベルは `.tflite` ファイルメタデータから取得できます。

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download(tflite_file)

[Android](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/android/) および [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/ios) では、TFLite 音声モデルを使用して推論を実行する、エンドツーエンドのサンプルアプリを確認できます。